In [114]:
import chardet

with open('data_for_spam.csv', 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
#splitting thr csv file into A and y

In [115]:
import pandas as pd

dataset = pd.read_csv('data_for_spam.csv', encoding=result['encoding'])

In [116]:
#splitting thr csv file into A and y

x = dataset.iloc[:, 0]  #Fetures (Message)
y = dataset.iloc[:, 1]  # Target (Spam or Ham)

X = x.to_dict()  # converting the features into dictionary
X

{0: 'WINNER!! As a valued network customer you have been selected to receivea ?900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only',
 1: 'okmail: Dear Dave this is your final notice to collect your 4* Tenerife Holiday or #5000 CASH award! Call 09061743806 from landline. TCs SAE Box326 CW25WX 150ppm',
 2: '07732584351 - Rodger Burns - MSG = We tried to call you re your reply to our sms for a free nokia mobile + free camcorder. Please call now 08000930705 for delivery tomorrow',
 3: 'URGENT! This is the 2nd attempt to contact U!U have WON ?1000CALL 09071512432 b4 300603t&csBCM4235WC1N3XX.callcost150ppmmobilesvary. max?7. 50',
 4: 'Congrats! Nokia 3650 video camera phone is your Call 09066382422 Calls cost 150ppm Ave call 3mins vary from mobiles 16+ Close 300603 post BCM4284 Ldn WC1N3XX',
 5: 'URGENT This is our 2nd attempt to contact U. Your ?900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893',
 6: 'Want explicit SEX in 30 

In [117]:
# train test split 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)


In [118]:
# Index ascending
y_train = y_train.reset_index().iloc[:, 1]

y_train

0      ham
1      ham
2      ham
3      ham
4     spam
      ... 
63    spam
64    spam
65     ham
66     ham
67    spam
Name: labels, Length: 68, dtype: object

In [119]:
#class wise data entry in training_data
training_data = []
for d in range(len(X_train)):
    #spam class training_data
    if y_train[d] == "spam":
        training_data.append({"class": "spam", "sentence": X_train[d]})
    #ham class training_data 
    else:
        training_data.append({"class": "ham", "sentence": X_train[d]})
#length of thr trainig_data  
print("%s sentences of training data" % len(training_data))

68 sentences of training data


In [120]:
# class dictitem
class_words = {}

# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

class_words

{'ham': [], 'spam': []}

In [121]:
import numpy as np
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from collections import Counter

stop_words = set(stopwords.words('english'))
stemmer = LancasterStemmer()

# Spam and Ham word list creat
sp_word = []
hm_word = []
cr_words = []
for data in training_data:
    sen = data['sentence']

    # temporarry list for appending to word list
    cr_words = []

    print("---------------------------------- ")
    # removing digit and punctuation
    sentence = re.sub(r'\d+', '', str(sen))
    print("4. " + sentence)
    sentence = re.sub('[' + string.punctuation + ']', '', sentence)
    print("5. " + sentence)
    print("---------------------------------- ")

    # Spam word lists
    if data['class'] == 'spam':
        for word in nltk.word_tokenize(sentence):
            if word not in stop_words:
                stemmed_word = stemmer.stem(word.lower())
                cr_words.append(stemmed_word)

            # Count occurrences of each word in cr_words
            count = Counter(cr_words)

            # Sort words by count in descending order and get only the words
            sorted_words = [word for word, freq in count.most_common()]
            sorted_words_array = np.array(sorted_words)

            print("8. ")
            print(sorted_words_array)

            # Form a spam word list
            hm_word.append(sorted_words_array)
    else:
        #spam word lists
        for word in nltk.word_tokenize(sentence):
            if word not in stop_words:
                stemmed_word = stemmer.stem(word.lower())
                cr_words.append(stemmed_word)
                print("6. " + stemmed_word)
                print("---------------------------------- ")

        count = Counter(cr_words)
        sorted_words = [word for word, _ in count.most_common()]
        sorted_words_array = np.array(sorted_words)

        print("8.")
        print(sorted_words_array)

        # Form a spam word list
        hm_word.append(sorted_words_array)


---------------------------------- 
4. I'm going  lunch now wif my family then aft dat i go str  orchard lor.
5. Im going  lunch now wif my family then aft dat i go str  orchard lor
---------------------------------- 
6. im
---------------------------------- 
6. going
---------------------------------- 
6. lunch
---------------------------------- 
6. wif
---------------------------------- 
6. famy
---------------------------------- 
6. aft
---------------------------------- 
6. dat
---------------------------------- 
6. go
---------------------------------- 
6. str
---------------------------------- 
6. orchard
---------------------------------- 
6. lor
---------------------------------- 
8.
['im' 'going' 'lunch' 'wif' 'famy' 'aft' 'dat' 'go' 'str' 'orchard' 'lor']
---------------------------------- 
4. HOW ARE U? I HAVE MISSED U! I HAVENT BEEN UP  MUCH A BIT BORED WITH THE HOLIDAY WANT  GO BAK  COLLEGE! SAD ISNT IT?xx
5. HOW ARE U I HAVE MISSED U I HAVENT BEEN UP  MUCH A BIT BORED WIT

In [122]:
#forming a ham matrix
for i in range(1):
    #total list of ham_row in a column then find the max length a which will be the row dimention
    a = []
    cl_word = hm_word
    hm_word = []
    for j in range(len(cl_word)):
        a.append(len(cl_word[j]))
    for j in range(len(cl_word)):
        n = cl_word[j].tolist()

        #finding highest row length
        w = max(a)
        for k in range(w):
            if k == len(cl_word[j]):
                b = len(cl_word[j]) + 1
                #append rest of by appending to
                for l in range(b, w + 1):
                    #to added cause its  in stop word 
                    n.append('None')
        hm_word.append(n)
hm_word_matrix = np.array(hm_word)

In [124]:

#forming a spam matrix
for i in range(1):
    #total list of spam_row in a column then find the max length a which will be the row dimention
    a = []
    cl_word = sp_word
    sp_word = []
    for j in range(len(cl_word)):
        a.append(len(cl_word[j]))
    for j in range(len(cl_word)):
        n = cl_word[j].tolist()
        #finding highest row length
        w = max(a)
        for k in range(w):
            if k == len(cl_word[j]):
                b = len(cl_word[j]) + 1
                #append rest of by appending to
                for l in range(b, w + 1):
                    n.append('None')
        sp_word.append(n)
    #convert list to array
sp_word_matrix = np.array(sp_word)
###end forming the spam word matrix


In [126]:
#smoothing factor
f = .15

In [128]:

#states calculaton for ham_word_matrix 
hm_obs = 0
for i in range(len(hm_word)):
    a = len(hm_word[i])
    for j in range(a):
        hm_obs += 1
hm_states = int((hm_obs / len(hm_word)))


In [136]:
#states calculaton for spam_word_matrix 
sp_obs = 0
for i in range(len(sp_word)):
    a = len(sp_word[i])
    for j in range(a):
        sp_obs += 1
if (len(sp_word) == 0):
    sp_states = 0
else:
    sp_states = int((sp_obs / len(sp_word)))

In [137]:

#ith sate probabilty for any classes
def state_ith_prob(wd, cr_word, cr_word_matrix, state):
    c_R_d = []
    count_R_d = 0
    for i in range(1):
        a = len(cr_word)
        for k in range(a):
            if wd == cr_word_matrix[k][state]:
                c_R_d.append(1)
            count_R_d += 1
    return len(c_R_d), count_R_d


#n_states probability
def state_tot_prob(wd, cr_word, cr_word_matrix):
    c_A_d = []
    count_A_d = 0
    for i in range(len(cr_word)):
        a = len(cr_word[i])
        for k in range(a):
            if wd == cr_word_matrix[i][k]:
                c_A_d.append(1)
            count_A_d += 1
    return len(c_A_d), count_A_d


#hmm model
def hmm(w, cl, state, cr_word, cr_word_matrix):
    if cl == 'spam':
        R = state_ith_prob(w, cr_word, cr_word_matrix, state)
        A = state_tot_prob(w, cr_word, cr_word_matrix)
        R_d, R_ln = R
        A_d, A_ln = A
        score = (f * (R_d / R_ln) + (1 - f) * (A_d / A_ln))
    else:
        R = state_ith_prob(w, cr_word, cr_word_matrix, state)
        A = state_tot_prob(w, cr_word, cr_word_matrix)
        R_d, R_ln = R
        A_d, A_ln = A
        score = (f * (R_d / R_ln) + (1 - f) * (A_d / A_ln))
    return score


In [140]:

# transition matrix for ham
# y is transition list
y = []
for i in range(hm_states):
    # row list appended in y
    a = []
    for j in range(hm_states):
        # i=j+1 emitted
        if j == i + 1:
            a.append(1)
        else:
            a.append(0)
    y.append(a)
# y_list to array
hm_transition_matrix = np.array(y)

In [142]:

# transition matrix for spam
# y is transition list
y = []
for i in range(sp_states):
    # row list appended in y
    a = []
    for j in range(sp_states):
        # i=j+1 emitted
        if j == i + 1:
            a.append(1)
        else:
            a.append(0)
    y.append(a)
# y_list to array
sp_transition_matrix = np.array(y)

In [143]:
# learning_Rate
n = 10000

# classify the test sentences

print("9. " + sentence)

9. Had your mobile  months or more U R entitled to Update to the latest colour mobiles with camera for Free Call The Mobile Update Co FREE on 


In [144]:

def classify(sentence):
    sentence = re.sub(r'\d+', '', sentence)
    sentence = re.sub('[' + string.punctuation + ']', '', sentence)
    sentence = sentence.lower()
    tokens = []
    for word in nltk.word_tokenize(sentence):
        if word not in stop_words:
            stemmed_word = stemmer.stem(word.lower())
            tokens.append(stemmed_word)

    high_class = None
    high_score = 0
    # loop through our classes
    for c in classes:
        # calculate score of sentence for each class
        if c == 'ham':
            alpha = 0
            sum = 0
            # header loop for calculating alpha_T
            for i in range(hm_states):
                # individual word in tokens and alpha_T count
                for word in tokens:
                    # word probabilty
                    a = hmm(word, c, i, hm_word, hm_word_matrix)
                    #                    print(a)
                    # sates probabilty
                    for j in range(hm_states):
                        # at first state initial probability 1
                        if 0 == i:
                            r = 1
                        # r after state one states probability
                        else:
                            for j in range(hm_states):
                                # emitted only i=j+1
                                sum = sum + b * (hm_transition_matrix[j][i])
                            r = sum
                    # o1,o2,......oT probability
                    b = r * a
                    # i_th probability of a sentence
                    # alpha_T at the end
                    alpha = alpha + b
            score = alpha
            score = n * (score)
        else:
            c == 'spam'
            alpha = 0
            sum = 0
            for i in range(sp_states):
                for word in tokens:

                    a = hmm(word, c, i, sp_word, sp_word_matrix)

                    for j in range(hm_states):
                        if 0 == i:
                            r = 1
                        else:
                            for j in range(sp_states):
                                sum = sum + b * sp_transition_matrix[j][i]
                            r = sum
                    b = r * a
                    alpha = alpha + b
            # score global variable for comparision
            score = alpha
            # n smoothing factor
            score = n * (score)
            # determine class according to score
        if score > high_score:
            high_class = c
            high_score = score
    return high_class, high_score


In [146]:

# y_test calculation
z = []
for j in range(len(X_test)):
    print(X_test[j])
    z.append(classify(X_test[j]))

    print(z)

# list to series
Z = pd.Series((v[0] for v in z))

y_pred = Z
# confusion metrix
# print("10. " + y_test)
# print("11. " + y_pred)

8007 FREE for 1st week! No1 Nokia tone 4 ur mob every week just txt NOKIA to 8007 Get txting and tell ur mates www.getzed.co.uk POBox 36504 W4 5WQ norm 150p/tone 16+
[('ham', 245.15190691661272)]
WOT STUDENT DISCOUNT CAN U GET ON BOOKS?
[('ham', 245.15190691661272), ('ham', 78.40982546864899)]
Play w computer? Aiyah i tok 2 u lor?
[('ham', 245.15190691661272), ('ham', 78.40982546864899), ('ham', 1027.276910131011)]
BOO BABE! U ENJOYIN YOURJOB? U SEEMED 2 B GETTIN ON WELL HUNNY!HOPE URE OK?TAKE CARE & I?LLSPEAK 2U SOONLOTS OF LOVEME XXXX.
[('ham', 245.15190691661272), ('ham', 78.40982546864899), ('ham', 1027.276910131011), ('ham', 299.45054945054943)]
Yup i thk so until e shop closes lor.
[('ham', 245.15190691661272), ('ham', 78.40982546864899), ('ham', 1027.276910131011), ('ham', 299.45054945054943), ('ham', 14.097240357249778)]
We not watching movie already. Xy wants 2 shop so i'm shopping w her now.
[('ham', 245.15190691661272), ('ham', 78.40982546864899), ('ham', 1027.276910131011),

In [148]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

cm = confusion_matrix(y_test, y_pred)
Accuracy_Score = accuracy_score(y_test, y_pred)

print("Accuracy_Score = ", Accuracy_Score)

Precision = precision_score(y_test, y_pred, average='weighted')
print("Precision= ", Precision)

Accuracy_Score =  0.6666666666666666
Precision=  0.4444444444444444


C:\Users\agexh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
